##  **Import libraries**

In [52]:
import pandas as pd
import pyodbc as odbc

## **Extract table data**

In [53]:
cat_data = pd.read_html('https://en.wikipedia.org/wiki/List_of_cat_breeds')

In [54]:
cat_df = cat_data[1]

In [55]:
cat_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 102 entries, 0 to 101
Data columns (total 7 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Breed                 102 non-null    object 
 1   Location of origin    102 non-null    object 
 2   Type                  102 non-null    object 
 3   Body type             101 non-null    object 
 4   Coat type and length  102 non-null    object 
 5   Coat pattern          101 non-null    object 
 6   Image                 0 non-null      float64
dtypes: float64(1), object(6)
memory usage: 5.7+ KB


## **Data cleanup**

In [56]:
# Drop the image column
cat_df.drop(['Image'], axis=1, inplace=True)

# Remove any brackets and numbers from the 'Breed' column
cat_df['Breed'] = cat_df['Breed'].str.replace(r'\[.*\]', '', regex=True).str.strip()

# Remove any unwanted characters from 'Location of origin'
cat_df['Location of origin'] = cat_df['Location of origin'].str.replace('\[.*\]', '', regex=True).str.strip()

# Drop rows with missing values (empty strings in this case)
cat_df = cat_df.replace('', pd.NA).dropna()

# Reset the index after dropping rows
cat_df = cat_df.reset_index(drop=True)


## **Show data**

In [57]:
cat_df

,Breed,Location of origin,Type,Body type,Coat type and length,Coat pattern
0,Abyssinian,"Unspecified, but somewhere in Afro-Asia, likel...",Natural,Semi-foreign,Short,Ticked tabby
1,Aegean,Greece,Natural,Moderate,Semi-long,Multi-color
2,American Bobtail,United States,Mutation of shortened tail,Cobby,Semi-long,All
3,American Curl,United States,Mutation,Semi-foreign,Semi-long,All
4,American Ringtail,United States,Mutation,Foreign,Semi-long,All
...,...,...,...,...,...,...
96,Turkish Angora,Turkey,Natural,Semi-cobby,Semi-long,All
97,Turkish Van,Developed in United Kingdom; foundation stock ...,Natural,Semi-cobby,Semi-long,Van pattern
98,Turkish Vankedisi (white variety of Turkish Van),"Lake Van, Turkey",Natural,Svelte,Long,Solid white
99,Ukrainian Levkoy,Ukraine,Crossbreed between the Donskoy and Scottish Fold,Moderate,Hairless,Solid gray


## **Save data to SQL table**

In [58]:
dsn = 'catDB'

# Create a connection string using the DSN
connection_string = f"""
    DSN={dsn};
    Trusted_Connection=yes;
"""

# Create a connection
conn = odbc.connect(connection_string)

# Create a cursor
cursor = conn.cursor()

# Insert Data into SQL table
insert_query = '''
    INSERT INTO CatBreeds (Breed, [Location of origin], Type, [Body type], [Coat type and length], [Coat pattern])
    VALUES (?, ?, ?, ?, ?, ?)
'''

# Assuming you have a DataFrame named 'cat_df' with the specified columns
for index, row in cat_df.iterrows():
    cursor.execute(insert_query, (row['Breed'], row['Location of origin'], row['Type'], row['Body type'], row['Coat type and length'], row['Coat pattern']))

# Commit the changes
conn.commit()

# Close the cursor and connection
cursor.close()
conn.close()

print(f"Data saved to SQL Database")

Data saved to SQL Database
